# Final Project

## 1. Scope the Project

Our plan is to use a supervised learning technique to predict which customers are most likely to cancel their subscription using **the past three months of customer data which includes subscription and listening history**.

## 2. Gather Data

Read the following files into Python:
* Customer data: *maven_music_customers.csv*
* Listing history: *maven_music_listening_history.xlsx*

In [ ]:
# Read in the customer data

import pandas as pd
import numpy as np
import seaborn as sns

df_customers = pd.read_csv('/home/ashkan/ASHKAN/Projects/DS_Projects/0_Maven_Music/maven_music_customers.csv')
df_customers.head()


In [ ]:
# Read in the listening history

df_history = pd.read_excel('/home/ashkan/ASHKAN/Projects/DS_Projects/0_Maven_Music/maven_music_listening_history.xlsx')
df_history.head()

In [ ]:
# Hint: Where might you find listening history data beyond the ID's?
# >>> other sheets in the Excel file!

In [ ]:
# Read in the audio data
df_audio = pd.read_excel('/home/ashkan/ASHKAN/Projects/DS_Projects/0_Maven_Music/maven_music_listening_history.xlsx', sheet_name=1)
df_audio.head()

In [ ]:
# Read in the session data
df_sessions = pd.read_excel('/home/ashkan/ASHKAN/Projects/DS_Projects/0_Maven_Music/maven_music_listening_history.xlsx', sheet_name=2)
df_sessions.head()

## 3. Clean Data

### a. Convert Data Types

Check the data types of the data in the tables and convert to numeric and datetime values as necessary.

In [ ]:
# Check the data types
df_customers.dtypes

In [ ]:
df_audio.dtypes

In [ ]:
df_history.dtypes

In [ ]:
df_sessions.dtypes

In [ ]:
# Convert objects to numeric and datetime fields
print(df_customers.dtypes)
df_customers.head()

In [ ]:
df_customers['Member Since'] = pd.to_datetime(df_customers['Member Since'], format="%m/%d/%y")
df_customers['Subscription Rate'] = pd.to_numeric(df_customers['Subscription Rate'].str.replace('$', ''))
df_customers['Cancellation Date'] = pd.to_datetime(df_customers['Cancellation Date'], format="%m/%d/%y")

In [ ]:
df_customers.dtypes

### b. Resolve Data Issues

Check for missing data, inconsistent text and typos, duplicate data and outliers.

#### i. Missing Data

In [ ]:
# Look for NaN values in the data
for df in [df_customers, df_audio, df_history, df_sessions]:
    print(df.isna().sum(), '\n')

So we need to fix just the df_customers.

In [ ]:
df_customers.sample(5)

In [ ]:
# Nans in Subscription Plan:

df_customers[['Subscription Rate', 'Subscription Plan']].drop_duplicates()

So, we need to convert all NaNs in the Plan to the Basic (Ads).
Also, that 99.99 should be typo and 9.99 would be correct.

In [ ]:
df_customers.fillna({'Subscription Plan': 'Basic (Ads)'}, inplace=True)

In [ ]:
# Nans in Discount?:

df_customers[['Subscription Rate', 'Discount?']].drop_duplicates()

It seems that 9.99 (and 2.99) would be No discount and 7.99 would be with discount.

In [ ]:
df_customers[df_customers['Subscription Rate'] == 7.99][['Subscription Rate','Discount?']]

All 7.99s are already yes, so we can safely replace all others with no.

In [ ]:
# We can do the following:
# df_customers['Discount?'].fillna('No', inplace=True)

# but better than that:
df_customers['Discount?'] = np.where(df_customers['Discount?'] == 'Yes', 1, 0)

Nans in Cancellation Date:

It seems that all the Nans (Not a time) means that they have not yet cancelled. So leave that column.

#### ii. Inconsistent Text & Typos

In [ ]:
# Look for inconsistent text & typos
df_customers.describe().T

It's time to take care of the Subscription Rate of 99.99

In [ ]:
df_customers[df_customers['Subscription Rate'] > 7.99]

In [ ]:
df_customers.iloc[15, 5] = 9.99

In [ ]:
df_history.describe().T

In [ ]:
df_history.head()

In [ ]:
df_history['Audio Type'].value_counts()

In [ ]:
df_audio.describe().T

In [ ]:
df_audio.head()

In [ ]:
df_audio['Genre'].value_counts()

Let's change all Pop Musics to Pop that we already have it.

In [ ]:
df_audio['Genre'] = pd.DataFrame(
    np.where(df_audio['Genre'] == 'Pop Music', 'Pop', df_audio.Genre)
    )

In [ ]:
df_audio['Genre'].value_counts()

In [ ]:
df_sessions.describe()

In [ ]:
df_sessions.head()

In [ ]:
df_sessions.describe()

#### iii. Duplicate Rows

In [ ]:
# Look for duplicate rows
for df in [df_customers, df_audio, df_history, df_sessions]:
    print(df[df.duplicated()])

#### iv. Outliers

Look for outliers:
We already did df.describe for all dfs. We fixed the 99.99 and there is no other suspicious value.


### c. Create New Columns

Create two new columns that will be useful for EDA and modeling:
* Cancelled: whether a customer cancelled or not
* Email: Remove the "Email:" from the email addresses

In [ ]:
# Create a 'Cancelled' column
df_customers.head()


In [ ]:
df_customers['Cancelled'] = np.where(
    df_customers['Cancellation Date'].isna(), 0, 1
)
df_customers.head()

In [ ]:
# Create an updated 'Email' column without the Email: portion
df_customers['Email'] = df_customers['Email'].str.replace('Email: ', '')
df_customers.head()

## 4. EDA

Try to better understand the customers who cancelled:
* How long were they members before they cancelled?
* What percentage of customers who cancelled had a discount vs customers who didn't cancel?

In [ ]:
# How long were customers members before they cancelled?
df_customers.head()


In [ ]:
df_customers[df_customers['Cancelled'] == 1].sample(3)

How long were they members before they cancelled?

In [ ]:
(df_customers['Cancellation Date'] - df_customers['Member Since']).mean()

The number is low and that is because we only have data for 3 months. We need more!

Customers are divided as:
1. Discount+  
    1.1. Cancelled +  
    1.2. Cancelled -  
2. Discount-  
    2.1. Cancelled +  
    2.2. Cancelled -  

In [ ]:
# Cancellation rate for those who had a discount (ratio of: 1.1/1.2)
(
    (df_customers[df_customers['Discount?']==1]['Cancelled'].sum())
    /
    (df_customers[df_customers['Discount?']==1]['Cancelled'].count())
)

In [ ]:
# Cancellation rate for those who did not have a discount (ratio of: 2.1/2.2)
(
    (df_customers[df_customers['Discount?']==0]['Cancelled'].sum())
    /
    (df_customers[df_customers['Discount?']==0]['Cancelled'].count())
)


It kinda makes sense: people who got discount, are ther for only the discount.

In [ ]:
# Visualize the cancellation rate for those with a discount vs those without a discount
sns.barplot(
    data=pd.DataFrame(
        [['With Discount', 0.857],
        ['No Discount', 0.304]],
        columns=['Type', 'Rate']
    ),
    x='Rate', hue='Type'
)


Better understand the customers' listening histories:
* Join together the listening history and audio tables
* How many listening sessions did each customer have in the past 3 months?
* What were the most popular genres that customers listened to?

In [ ]:
# Split the ID in the audio data so the column can be joined with other tables


In [ ]:
# Hint: Check the data type of Audio ID in the audio table


In [ ]:
# The number of listening sessions that each customer had in the past 3 months


In [ ]:
# The most popular genres that customers listened to


## 5. Prep for Modeling

Create a DataFrame that is ready for modeling with each row representing a customer and the following numeric, non-null columns:
* Customer ID
* Whether a customer cancelled or not
* Whether a customer received a discount or not
* The number of listening sessions
* Percent of listening history consisting of Pop
* Percent of listening history consisting of Podcasts

In [ ]:
# Create a dataframe ready for modeling


In [ ]:
# Calculate the number of listening sessions for each customer


In [ ]:
# Percent pop


In [ ]:
# Percent podcasts


Visualize the relationships in the modeling DataFrame using a pair plot:
* What are some of your observations?
* What variables might do a good job predicting customer cancellation?